In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import random as rn
import itertools as its
from sklearn.preprocessing import normalize
from tqdm import tqdm
# theta1 = 2
# theta2 = 1.6
# theta3 = 0.8
# theta4 = 1.7
#theta_true = np.array([theta1,theta2,theta3,theta4])
# def rw(s,a1,a2, theta = theta_true): # sc in {0,1}, a1,a2 in {0,1}
#   return theta[0] * a1 + theta[1]*a2 + theta[2]*(1-a1+1-a2) - theta[3]*(s+1)
# #print(rw(2,1,1))

theta1 = 1
theta2 = 2
theta_true = np.array([theta1,theta2])

def rw(s,a1,a2, theta = theta_true): # sc in {0,1}, a1,a2 in {0,1}
  if s ==0:
    if a1==a2:
      return theta[0]
    else:
      return 0
  else:
    if a1==a2:
      return 0
    else:
      return theta[1]
  # return theta[0] * a1 + theta[1]*a2 + theta[2]*(1-a1+1-a2) - theta[3]*(s+1)
#print(rw(1,1,1))



dygame = np.zeros([2,2,2,2,2]) #st,at1, at2,st+1, zt+1
ind = 0
for st, at1, at2 in its.product(range(2),range(2),range(2)):
  ind = ind + 1
  rn.seed(ind)
  prob = rn.choices(np.arange(11), weights=None, cum_weights=None, k=4)
  dygame[st,at1,at2,:,:] = np.reshape(prob/np.sum(prob),[2,2])
#print(dygame)

def siglam(z,x,a1,a2,theta1=dygame):
  theta1 = np.reshape(theta1,[2,2,2,2,2])
  lambd = theta1[:,a1,a2,:,z].T.dot([x,1-x])
  #print(lambd)
  sigma = np.sum(lambd)
  return lambd[0]/sigma, sigma
#print(siglam(0,[1/2,1/2],1,0))

gamma = 0.5772
beta = 0.9
def PerfectMarkov(iniCCP1, iniCCP2,theta = theta_true, maxiter = 200,blief = 11):
  
  x = np.linspace(0,1,blief)
  
  ccp1 = iniCCP1
  ccp2 = iniCCP2
  
  #for i in tqdm(range(maxiter)):
  for i in range(maxiter):

    ccp1_old = ccp1.copy()
    ccp2_old = ccp2.copy()
    
    Q1 = np.zeros([blief,2])
    Q2 = np.zeros([blief,2])
    
    for j in range(maxiter):
      Q1_old = Q1.copy()
      Q2_old = Q2.copy()
      
      #print(j,haha1) 
      
      V1 = gamma + np.log(np.sum(np.exp(Q1),axis=1))
      V2 = gamma + np.log(np.sum(np.exp(Q2),axis=1))
      
      for indx, inda in its.product(x,range(2)):
        
        l1 = np.zeros([2,2])#z,a2
        s1 = np.zeros([2,2])
        V1_tep = np.zeros([2,2])
        
        l2 = np.zeros([2,2])#z,a1
        s2 = np.zeros([2,2])
        V2_tep = np.zeros([2,2])
        for indl, inds in its.product(range(2),range(2)): #z , a2
          #print(siglam(indl,indx,inda,inds))
          l1[indl,inds],s1[indl,inds] = siglam(indl,indx,inda,inds)
          fl = int(np.floor(l1[indl,inds] * (blief-1)))
          if fl == blief-1:
            V1_tep[indl,inds] = V1[fl]
          else: 
            V1_tep[indl,inds] = V1[fl] * (l1[indl,inds]*(blief-1) - fl) + V1[fl+1]* (1 + fl -l1[indl,inds]*(blief-1)) 
          
          l2[indl,inds],s2[indl,inds] = siglam(indl,indx,inds,inda) #z , a1
          fl = int(np.floor(l2[indl,inds] * (blief-1)))
          if fl == blief-1:
            V2_tep[indl,inds] = V2[fl]
          else: 
            V2_tep[indl,inds] = V2[fl] * (l2[indl,inds]*(blief-1) - fl) + V2[fl+1]* (1 + fl -l2[indl,inds]*(blief-1)) 
        
        #print(indl)
        V1_tep = V1_tep * s1
        V2_tep = V2_tep * s2
        
        ind = int(indx *(blief-1))
        Q1[ind,inda] = ccp2[ind,0] *( indx* rw(0,inda,0,theta = theta) + (1-indx) * rw(1,inda,0,theta = theta) + \
                          beta * np.sum(V1_tep[:,0])) +\
                        ccp2[ind,1] * (indx* rw(0,inda,1,theta = theta) + (1-indx) * rw(1,inda,1,theta = theta) +\
                          beta * np.sum(V1_tep[:,1]))
                        
        Q2[ind,inda] = ccp1[ind,0] *( indx* rw(0,0,inda,theta = theta) + (1-indx) * rw(1,0,inda,theta = theta) + \
                          beta * np.sum(V2_tep[:,0])) +\
                        ccp1[ind,1] * (indx* rw(0,1,inda,theta = theta) + (1-indx) * rw(1,1,inda,theta = theta) +\
                          beta * np.sum(V2_tep[:,1]))
                        
      #print(j,haha1)
      if np.max(np.abs(Q1-Q1_old)) <= 0.01 and np.max(np.abs(Q2-Q2_old))<=0.01:
        #print(i,j)
        #print(j,np.max(Q1),np.max(Q2),np.max(np.abs(Q1-Q1_old)),np.max(np.abs(Q2-Q2_old)))
        break
      #print(j,Q1,Q2)
    ccp1 = normalize(np.exp(Q1), norm='l1',axis=1)#np.exp(Q1)/(np.sum(np.exp(Q1),axis=1))
    ccp2 = normalize(np.exp(Q2), norm='l1',axis=1)#np.exp(Q1)/(np.sum(np.exp(Q1),axis=1))
    
    if np.max(np.abs(ccp1-ccp1_old))<=1e-4 and np.max(np.abs(ccp2-ccp2_old))<=1e-4:
      #print(i,ccp1,ccp2)
      #print(i,np.max(ccp1),np.max(ccp2),np.max(np.abs(ccp1-ccp1_old)),np.max(np.abs(ccp2-ccp2_old)))
      break
    #print(i,np.max(ccp1),np.max(ccp2),np.max(np.abs(ccp1-ccp1_old)),np.max(np.abs(ccp2-ccp2_old)))
  if i == maxiter -1:
    print('fail to converge')
  return ccp1, ccp2, i

In [2]:
blief = 11
haha1 = np.ones([blief,2])/2
ccp1, ccp2,_ = PerfectMarkov(haha1,haha1,blief= blief)
print(ccp1.shape,ccp2.shape)

(11, 2) (11, 2)


In [3]:
x = np.linspace(0,1,blief)

N = 1000
T = 20
st = np.zeros([N,T+1],dtype = int)
zt = np.zeros([N,T],dtype = int)
a1t = np.zeros([N,T],dtype = int)
a2t = np.zeros([N,T],dtype = int)
belief = 1/2 * np.ones([N,T+1])

rn.seed(0)
st[:,0] = rn.choices([0,1],[1/2,1/2], k = N)

for ss in range(N):
  for t in range(T):
    fl = int(np.floor(belief[ss,t] * (blief-1)))
    if fl == blief-1:
      prob1 = ccp1[fl,:]
      prob2 = ccp2[fl,:]
    else: 
      prob1 = ccp1[fl,:] * (belief[ss,t] *(blief-1) - fl) + ccp1[fl+1,:]* (1 + fl -belief[ss,t]*(blief-1))
      prob2 = ccp2[fl,:] * (belief[ss,t] *(blief-1) - fl) + ccp2[fl+1,:]* (1 + fl -belief[ss,t]*(blief-1))
    
    a1t[ss,t] = rn.choices([0,1],prob1)[0]
    a2t[ss,t] = rn.choices([0,1],prob2)[0]

    prob3 = dygame[st[ss,t], a1t[ss,t],a2t[ss,t],:,:]#st,at1, at2,st+1, zt+1

    st[ss,t+1], zt[ss,t] = rn.choices([[0,0],[0,1],[1,0],[1,1]], np.reshape(prob3,[4,1]))[0]
    belief[ss,t+1],_ = siglam(zt[ss,t],belief[ss,t],a1t[ss,t],a2t[ss,t])

In [4]:
print(st[0:2,0:10])
print(belief[0:2,0:10])
print(a1t[0:2,0:10])
print(a2t[0:2,0:10])
print(zt[0:2,0:10])

[[1 1 1 0 0 1 0 1 0 0]
 [1 1 1 0 1 0 0 1 1 1]]
[[0.5        0.56818182 0.31127049 0.7286058  0.88941998 0.1453459
  0.58403211 0.66067487 0.55610595 0.37922205]
 [0.5        0.56818182 0.92534381 0.51899367 0.38545842 0.30407851
  0.72592714 0.35673611 0.30590536 0.30487019]]
[[1 1 0 0 0 1 0 1 1 1]
 [1 0 1 1 0 0 1 1 1 0]]
[[1 0 1 1 0 1 0 1 1 1]
 [1 1 1 1 0 1 1 0 0 0]]
[[1 0 0 0 0 1 1 1 0 0]
 [1 1 1 0 0 0 0 0 0 0]]


In [5]:
##############Recover###############
#blief = 41
haha1 = np.ones([blief,2])/2
# theta = np.ones(4)
PolicyFunc = lambda theta_try : PerfectMarkov(haha1, haha1,theta = theta_try, maxiter = 200,blief = blief)
# PolicyFunc(theta_true)

def LogLike_MLE1(theta1,params=False,x0=belief[:,0],obz = zt, ac1 = a1t, ac2 = a2t):
  #print('Compute Perfect Markov Equlibrium')
  # ccp1,ccp2,_ = PolicyFunc(theta)
  loglike = 0
  blief_x = x0.copy()

  theta1_new = np.zeros([2,2,2,2,2])
  for i in range(2):
    for j in range(2):
      for k in range(2):
        ind_ijk = ((i*2+j)*2+k)*3
        #print(ind_ijk)
        theta1_new[i,j,k,0,0] = theta1[ind_ijk]
        theta1_new[i,j,k,0,1] = theta1[ind_ijk+ 1]
        theta1_new[i,j,k,1,0] = theta1[ind_ijk+ 2]
        theta1_new[i,j,k,1,1] = 1- theta1_new[i,j,k,0,0]-theta1_new[i,j,k,0,1]-theta1_new[i,j,k,1,0]

  #print('Compute Loglike')
  for ss in range(N):
    for t in range(T):
      # fl = int(np.floor(blief_x[ss] * (blief-1)))
      # if fl == blief-1:
      #   pi1 = ccp1[fl,ac1[ss,t]]
      #   pi2 = ccp2[fl,ac2[ss,t]]
      # else: 
      #   pi1 = ccp1[fl,ac1[ss,t]] * (blief_x[ss] *(blief-1) - fl) + ccp1[fl+1,ac1[ss,t]]* (1 + fl -blief_x[ss] *(blief-1)) 
      #   pi2 = ccp2[fl,ac2[ss,t]] * (blief_x[ss] *(blief-1) - fl) + ccp2[fl+1,ac2[ss,t]]* (1 + fl -blief_x[ss] *(blief-1))
      # loglike = loglike + np.log(pi1) + np.log(pi2)

      blief_x[ss],_ = siglam(obz[ss,t],blief_x[ss],ac1[ss,t],ac2[ss,t],theta1 = theta1_new)
      loglike = loglike + np.log(blief_x[ss])
  if params:
    print(-loglike)
  return -loglike

def LogLike_MLE2(theta2,theta1,params=True,x0=belief[:,0],obz = zt, ac1 = a1t, ac2 = a2t):
  #print('Compute Perfect Markov Equlibrium')
  ccp1,ccp2,_ = PolicyFunc(theta2)
  loglike = 0
  blief_x = x0.copy()
  #print('Compute Loglike')
  theta1_new = np.zeros([2,2,2,2,2])
  for i in range(2):
    for j in range(2):
      for k in range(2):
        ind_ijk = ((i*2+j)*2+k)*3
        #print(ind_ijk)
        theta1_new[i,j,k,0,0] = theta1[ind_ijk]
        theta1_new[i,j,k,0,1] = theta1[ind_ijk+ 1]
        theta1_new[i,j,k,1,0] = theta1[ind_ijk+ 2]
        theta1_new[i,j,k,1,1] = 1- theta1_new[i,j,k,0,0]-theta1_new[i,j,k,0,1]-theta1_new[i,j,k,1,0]

  for ss in range(N):
    for t in range(T):
      fl = int(np.floor(blief_x[ss] * (blief-1)))
      if fl == blief-1:
        pi1 = ccp1[fl,ac1[ss,t]]
        pi2 = ccp2[fl,ac2[ss,t]]
      else: 
        pi1 = ccp1[fl,ac1[ss,t]] * (blief_x[ss] *(blief-1) - fl) + ccp1[fl+1,ac1[ss,t]]* (1 + fl -blief_x[ss] *(blief-1)) 
        pi2 = ccp2[fl,ac2[ss,t]] * (blief_x[ss] *(blief-1) - fl) + ccp2[fl+1,ac2[ss,t]]* (1 + fl -blief_x[ss] *(blief-1))
      loglike = loglike + np.log(pi1) + np.log(pi2)

      blief_x[ss],_ = siglam(obz[ss,t],blief_x[ss],ac1[ss,t],ac2[ss,t],theta1=theta1_new)
  if params:
    print(theta2,-loglike)
  return -loglike
# def LogLike1_MLE(theta,x0=belief[:,0],obz = zt, ac1 = a1t, ac2 = a2t):
#   #print('Compute Perfect Markov Equlibrium')
#   ccp1,ccp2,_ = PolicyFunc(theta)
#   loglike = 0
#   blief_x = x0.copy()
#   #print('Compute Loglike')
#   for ss in range(N):
#     for t in range(T):
#       fl = int(np.floor(blief_x[ss] * (blief-1)))
#       if fl == blief-1:
#         pi1 = ccp1[fl,ac1[ss,t]]
#         pi2 = ccp2[fl,ac2[ss,t]]
#       else: 
#         pi1 = ccp1[fl,ac1[ss,t]] * (blief_x[ss] *(blief-1) - fl) + ccp1[fl+1,ac1[ss,t]]* (1 + fl -blief_x[ss] *(blief-1)) 
#         pi2 = ccp2[fl,ac2[ss,t]] * (blief_x[ss] *(blief-1) - fl) + ccp2[fl+1,ac2[ss,t]]* (1 + fl -blief_x[ss] *(blief-1))

#       blief_x[ss],sigma = siglam(obz[ss,t],blief_x[ss],ac1[ss,t],ac2[ss,t])
#       loglike = loglike + np.log(pi1) + np.log(pi2) + np.log(sigma)

#   return -loglike

In [6]:
print(theta_true,dygame)
true_dygame = np.reshape([[dygame[i,j,k,0,0], dygame[i,j,k,0,1], dygame[i,j,k,1,0]] for i in range(2) for j in range(2) for k in range(2)],[24,1])
print(-LogLike_MLE1(true_dygame,False))
print(-LogLike_MLE2(theta_true,true_dygame,False))


[1 2] [[[[[0.05       0.45      ]
    [0.4        0.1       ]]

   [[0.5        0.5       ]
    [0.         0.        ]]]


  [[[0.11764706 0.29411765]
    [0.23529412 0.35294118]]

   [[0.25       0.125     ]
    [0.5        0.125     ]]]]



 [[[[0.1875     0.25      ]
    [0.25       0.3125    ]]

   [[0.33333333 0.375     ]
    [0.20833333 0.08333333]]]


  [[[0.27272727 0.09090909]
    [0.63636364 0.        ]]

   [[0.1        0.5       ]
    [0.05       0.35      ]]]]]
-14271.993470960195
-27635.31312071463


In [82]:
# #print(dy1)
# #print(dy2)
# #print(dygame)
# #print(theta_true)
# #true_dygame = np.reshape([[dygame[i,j,k,0,0], dygame[i,j,k,0,1], dygame[i,j,k,1,0]] for i in range(2) for j in range(2) for k in range(2)],[24,1])
# true_dygame = np.reshape(dy2[:,:,:,0],[8,1])
# print(-LogLike_MLE1(true_dygame,False,tranStatue=True))
# print(-LogLike_MLE2(theta_true,true_dygame,False,tranStatue=True))
# #print(true_dygame)

In [7]:
from scipy.optimize import minimize
bounds = [[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]\
          ,[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]
from scipy.optimize import LinearConstraint

linear_constraint = LinearConstraint([[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1]], [0,0,0,0,0,0,0,0], [1,1,1,1,1,1,1,1])          
# resbrute1 = optimize.brute(LogLike_MLE1, rranges, full_output=True,
#                           finish=optimize.fmin)
x0 = 0.25*np.ones(24)

res = minimize(LogLike_MLE1, x0, method='trust-constr', #jac=rosen_der, hess=rosen_hess,
               constraints=[linear_constraint],
               options={'verbose': 3}, bounds=bounds)
theta1 = res.x
#Minimize a function over a given range by brute force.

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   |  25   |   0   | +1.3863e+04 | 1.00e+00 | 1.35e+03 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   0   |
|   2   |  50   |   1   | +9.0582e+03 | 7.00e+00 | 1.00e+03 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   2   |
|   3   |  75   |   6   | -4.4746e+03 | 2.78e+01 | 3.78e+02 | 1.92e-01 | 1.00e+00 |  1.00e-01   |   4   |
|   4   |  100  |  22   | +4.3525e+02 | 2.78e+01 | 1.14e+01 | 2.77e-03 | 8.20e+03 |  1.00e-01   |   4   |
|   5   |  125  |  42   | +1.9574e+02 | 2.78e+01 | 2.10e+00 | 0.00e+00 | 8.20e+03 |  1.00e-01   |   4   |
|   6   |  150  |  63   | +8.0512e+01 | 2.78e+01 | 1.15e+00 | 0.00e+00 | 8.20e+03 |  1.00e-01   |   4   |
|   7   |  175  |  84   | +4.8310e+01 | 2.78e+01 | 8.00e-01 | 0.00e+00 | 8.20e+03 |  1.00e-01   |   4   |
|   8   |  200  |  101  | +2.7031e+01 | 2.78e+

In [8]:
print(true_dygame,'\n',theta1)

[[0.05      ]
 [0.45      ]
 [0.4       ]
 [0.5       ]
 [0.5       ]
 [0.        ]
 [0.11764706]
 [0.29411765]
 [0.23529412]
 [0.25      ]
 [0.125     ]
 [0.5       ]
 [0.1875    ]
 [0.25      ]
 [0.25      ]
 [0.33333333]
 [0.375     ]
 [0.20833333]
 [0.27272727]
 [0.09090909]
 [0.63636364]
 [0.1       ]
 [0.5       ]
 [0.05      ]] 
 [5.34452966e-01 4.65547033e-01 5.66660179e-10 5.59388113e-01
 4.40611886e-01 5.53163851e-10 5.35435367e-01 4.64564632e-01
 2.39733024e-10 6.44763755e-01 3.55236245e-01 6.05941851e-10
 4.82006023e-01 5.17993968e-01 3.14382360e-09 5.26247695e-01
 4.73752267e-01 1.84954609e-08 5.52154131e-01 4.47845862e-01
 2.59973967e-09 4.91275241e-01 5.08724749e-01 3.56257422e-09]


In [9]:
#from scipy.optimize import brute
rranges = (slice(0, 3, 0.5), slice(0, 3, 0.5))
from scipy import optimize

func = lambda theta2: LogLike_MLE2(theta2,theta1)
#func = lambda theta2: LogLike_MLE2(theta2,true_dygame)
# resbrute = optimize.brute(func, rranges, full_output=True,
#                           finish=optimize.fmin)

bounds = [[0,3],[0,3]]
resbrute = minimize(func, [1,2], method='L-BFGS-B',#'trust-constr', #options={'verbose': 3}, 
                    bounds=bounds)
#Minimize a function over a given range by brute force.

[1. 2.] 27652.525374446028
[1.00000001 2.        ] 27652.525374894063
[1.         2.00000001] 27652.525374097237
[0. 3.] 28010.170235669415
[1.e-08 3.e+00] 28010.170227404324
[0.         3.00000001] 28010.170240648575
[0.80976544 2.19023456] 27644.69062911453
[0.80976545 2.19023456] 27644.69062909896
[0.80976544 2.19023457] 27644.690629120876
[0.81610986 2.18677753] 27644.684235765468
[0.81610987 2.18677753] 27644.6842357651
[0.81610986 2.18677754] 27644.684235748515
[0.81682499 2.18965972] 27644.68123307443
[0.816825   2.18965972] 27644.681233076182
[0.81682499 2.18965973] 27644.681233068088
[0.81586156 2.1914563 ] 27644.680876739818
[0.81586157 2.1914563 ] 27644.680876732647
[0.81586156 2.19145631] 27644.680876738166
[0.81725886 2.19327499] 27644.67837193339
[0.81725887 2.19327499] 27644.67837192092
[0.81725886 2.193275  ] 27644.678371929982
[0.82284804 2.20054976] 27644.66835643841
[0.82284805 2.20054976] 27644.6683564311
[0.82284804 2.20054977] 27644.66835643843
[0.85179398 2.23350

In [10]:
resbrute#87

      fun: 27644.564790590302
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.45329216, -0.0865839 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 87
      nit: 10
   status: 0
  success: True
        x: array([0.90406444, 2.30659021])

In [11]:
#from scipy.optimize import brute
rranges = (slice(0, 3, 0.5), slice(0, 3, 0.5))
from scipy import optimize

#func = lambda theta2: LogLike_MLE2(theta2,theta1)
func = lambda theta2: LogLike_MLE2(theta2,true_dygame)
#resbrute = optimize.brute(func, rranges, full_output=True,
#                          finish=optimize.fmin)


bounds = [[0,3],[0,3]]
resbrute = minimize(func, [1,2], method='L-BFGS-B',#'trust-constr', #options={'verbose': 3}, 
                    bounds=bounds)
#Minimize a function over a given range by brute force.

[1. 2.] 27635.31312071463
[1.00000001 2.        ] 27635.313120687584
[1.         2.00000001] 27635.313120801547
[3. 0.] 29696.23797768165
[3.00000001 0.        ] 29696.23800437004
[3.e+00 1.e-08] 29696.237974630054
[1.03670446 1.96329554] 27635.811299576984
[1.03670447 1.96329554] 27635.81129975837
[1.03670446 1.96329555] 27635.811299385827
[1.00809697 1.99190303] 27635.266976302606
[1.00809698 1.99190303] 27635.266976322724
[1.00809697 1.99190304] 27635.26697632307
[1.00483804 1.98954808] 27635.25539880233
[1.00483805 1.98954808] 27635.255398823316
[1.00483804 1.98954809] 27635.255398824527
[0.99180233 1.98012827] 27635.212875165766
[0.99180234 1.98012827] 27635.212875182293
[0.99180233 1.98012828] 27635.21287518534
[0.87428802 1.89242368] 27635.067595305984
[0.87428803 1.89242368] 27635.067595287637
[0.87428802 1.89242369] 27635.067595325014
[0.88815291 1.89864443] 27635.06030198563
[0.88815292 1.89864443] 27635.06030198545
[0.88815291 1.89864444] 27635.06030198709
[0.88554395 1.8963

In [12]:
resbrute#39

      fun: 27635.060301977355
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.10513759,  0.14333636])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 39
      nit: 5
   status: 0
  success: True
        x: array([0.88812127, 1.89861624])